In [8]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 15 10:34:27 2018

@author: yuxi
"""

import matplotlib.pyplot as plt
import numpy as np 
from skimage.io import imread
from PIL import Image
import glob
import itertools
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils.vis_utils import plot_model

def newtxt(path_txt,path_image,crop_length,crop_width):
    # load the true demension of the image
    image = imread(path_image)
    true_length = image.shape[1]
    true_width = image.shape[0]
    
    #read the orginal demension of the image
    txtfile = open(path_txt)
    firstline = txtfile.readlines()[0].split(",") 
    original_length = int(firstline[2])
    original_width = int(firstline[3])
    txtfile.close()
    
    #read the number of the random points in the image
    txtfile = open(path_txt)
    count_points = int(txtfile.readlines()[5])
    txtfile.close()
    
    #read the corrdinate of the image
    txtfile = open(path_txt)
    data = txtfile.readlines()[6:6+count_points]
    corrdinate = np.zeros([count_points,2],dtype = np.int)
    for n in range(count_points): 
        data1 = data[n].split(",")
        corrdinate[n,0] =int(int(data1[0])*true_length/original_length)
        corrdinate[n,1] =int(int(data1[1])*true_width/original_width)
    txtfile.close()
    
    #read the label of each points and encode the label
    txtfile = open(path_txt)
    label_encode = np.zeros(count_points)
    label = txtfile.readlines()[6+count_points:6+count_points+count_points]
    for m in range(count_points):
        label1 = label[m].split(",")
        label2 = label1[1]
        new_l = label2.replace('\"', '')
        #coral
        if ((new_l=="Agalg") or (new_l=="Aga") or (new_l=="Agaf") or (new_l=="Col") or (new_l=="Helc") or (new_l=="Mdrc") or (new_l=="Mdrcb") or (new_l=="Mdrsd")or (new_l=="Mdrsf") or (new_l=="Man")or (new_l=="Mon")or (new_l=="Ocu")or (new_l=="Sco")or (new_l=="Sol")or (new_l=="Ste")or (new_l=="STY")):
            label_encode[m] = 0
        #dead coral plate
        elif (new_l=="DCP"):
            label_encode[m] = 1
        #rock
        elif (new_l=="ROC"):
            label_encode[m] = 2
        #red alage
        elif ((new_l=="CCA") or (new_l=="Amph") or (new_l=="Bot") or (new_l=="Haly") or (new_l=="Kal") or (new_l=="Mar") or (new_l=="PEY") or (new_l=="RHO") or (new_l=="RHbl")) :
            label_encode[m] = 3
        #green alage
        elif ((new_l=="Ana") or (new_l=="Cau") or (new_l=="Caup") or (new_l=="Caur") or (new_l=="Caus") or (new_l=="Chae") or (new_l=="CHL") or (new_l=="Cod") or (new_l=="Codin") or (new_l=="Hal") or (new_l=="Halc") or (new_l=="Hald") or (new_l=="Halt") or (new_l=="Micr") or (new_l=="Ulva") or (new_l=="Venv") or (new_l=="Verp")):
            label_encode[m] = 4 
        else:
            label_encode[m] = 5
    txtfile.close()
    #save the encode labeled pixel data into new files
    name_image = path_image.split("/")[2] 
    name_txt = name_image.split(".")[0]
    with open(str('2015-image-data - Copy/')+name_txt+str('.txt'), 'w+') as newtxtfile:
        newtxtfile.write(name_image+str('\n'))
        newtxtfile.write('x,y,label\n')
        for i in range(count_points):
            x = corrdinate[i,0]
            y = corrdinate[i,1]
            label = int(label_encode[i])
            newtxtfile.write('{0},{1},{2}\n'.format(x,y,label))
        newtxtfile.close()
    #transfer the pixel data into crop iamge data
    all_image = np.zeros([count_points,crop_length,crop_width,3],dtype=np.uint8)
    crop_x = int(crop_length/2)
    crop_y = int(crop_width/2)
    for i in range(count_points):
        if(corrdinate[i,0]-crop_x <0):
            corrdinate[i,0] = crop_x
        elif(corrdinate[i,1]-crop_y <0):
            corrdinate[i,1] = crop_y
        elif(corrdinate[i,0]+crop_x >true_length):
            corrdinate[i,0] = true_length-crop_x
        elif(corrdinate[i,1]+crop_y >true_width):
            corrdinate[i,1] =true_width-crop_y
        all_image[i,:,:,:] = image[corrdinate[i,1]-crop_y:corrdinate[i,1]+crop_y,corrdinate[i,0]-crop_x:corrdinate[i,0]+crop_x]
    return all_image, label_encode
#%%





#%%
def newimagedata(crop_length,crop_width):
    all_label = np.zeros(1)
    image_data = np.zeros([1,crop_length,crop_width,3],dtype=np.uint8)
    read_files = glob.glob("./2015-image-data - Copy/*.txt")
    #length = len(read_files)
    for name in read_files:
        name = name.split("\\")[1]
        name = name.split(".")[0]
        path_txt = str('./2015-image-data - Copy/')+name+str('.txt')
        path_image = str('./2015image/')+name+str('.jpg')
        new_image_data,label = newtxt(path_txt,path_image,crop_length,crop_width)
        all_label = np.hstack((all_label,label))
        image_data = np.vstack((image_data,new_image_data))
    final_data = image_data[1:,:,:,:]
    final_label = all_label[1:]
    return final_data,final_label
#%%
def create_plots(history):
    fig = plt.figure(figsize=(5,10))
    ax = fig.add_subplot(*[2,1,1])
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    ax.set_title('accuracy of CNN')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    #plt.savefig('accuracy.png')
    #plt.clf()
    #plt.show()
    
    ax = fig.add_subplot(*[2,1,2])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('loss of CNN')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    #plt.savefig('loss.png')
    #plt.clf()
    plt.show()
    
def plot_confusion_matrix(confusionmatrix, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        confusionmatrix = confusionmatrix.astype('float') / confusionmatrix.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(confusionmatrix)

    plt.imshow(confusionmatrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = confusionmatrix.max() / 2.
    for i, j in itertools.product(range(confusionmatrix.shape[0]), range(confusionmatrix.shape[1])):
        plt.text(j, i, format(confusionmatrix[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if confusionmatrix[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

#%%
def shuffle(data,label):
    data_shape = data.shape
    data = data.reshape(-1,data_shape[1]*data_shape[2]*data_shape[3])
    Z = np.column_stack((data,label))
    np.random.shuffle(Z)
    fully_data = Z[:,:,-1]
    shuffle_label = Z[:,-1]
    shuffle_data = fully_data.reshape(-1,30,30,3)
    return shuffle_data,shuffle_label
    
    
#%%six convolutional layers cnn model
def cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=(30,30,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))       
    
    
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25)) 
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6))
    model.add(Activation('softmax'))
    return model

#%%eight convolutional layers cnn model
def cnn_model2():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=(30,30,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))       
    
    
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25)) 
    
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6))
    model.add(Activation('softmax'))
    return model

#%%four convolutional layers
def cnn_model1():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=(30,30,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))       
    
    
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6))
    model.add(Activation('softmax'))
    return model

def read_generate_data(txtfolder_name,imagefloder_name,image_x,image_y):
    all_count = 1
    all_label = np.zeros(1,dtype=np.int)
    image_data = np.zeros([1,30,30,3],dtype=np.uint8)
    read_files = glob.glob(str('./')+txtfolder_name+str('/*.txt'))
    for name in read_files:
        name = name.split("\\")[1]
        name = name.split(".")[0]
        path_txt = str('./')+txtfolder_name+str('/')+name+str('.txt')
        path_image = str('./')+ imagefloder_name+str('/')+name+str('.jpg')
        new_image_data,label = generate_signal_imagedata(path_txt,path_image,image_x,image_y)
        all_label = np.hstack((all_label,label))
        image_data = np.vstack((image_data,new_image_data))
        print("Image {0} is finish ".format(all_count))
        all_count = all_count+1
    final_data = image_data[1:,:,:,:]
    final_label = all_label[1:]
    return final_data,final_label

def generate_signal_imagedata(path_txt,path_image,image_x,image_y):
    txt_file = open(path_txt)
    text = txt_file.readlines()[2:]
    count_points = len(text)
    
    crop_length = 30
    crop_width = 30
    all_image = np.zeros([count_points,crop_length,crop_width,3],dtype=np.uint8)
    label = np.zeros(count_points,dtype=np.int)
    crop_x = int(crop_length/2)
    crop_y = int(crop_width/2)
    image = imread(path_image)
    for i in range(count_points):
        text_piece = text[i]
        text_element = text_piece.split(',')
        l_x = int(float(text_element[0]))
        l_y = int(float(text_element[1]))
        label[i] = int(text_element[2])
        if(l_x-crop_x <0):
            l_x = crop_x
        if(l_y-crop_y <0):
            l_y = crop_y
        if(l_x+crop_x >image_y):
            l_x = image_y-15
        if(l_y+crop_y >image_x):
            l_y =image_x-15
        all_image[i,:,:,:] = image[l_y-15:l_y+15,l_x-15:l_x+15]
    txt_file.close()
    return all_image,label
#%%
#original_data,original_label = newimagedata(30,30)





